In [13]:
import pandas as pd
import os
import re
from importlib import reload
from helpers import constants; reload(constants)

In [4]:
raw_data_path = constants.RAW_TEXT_DIR
base_data_path = constants.BASE_DATA_DIR

In [7]:
df_posts = pd.read_pickle(base_data_path + '/post_df.pickle')

In [10]:
def split_into_sentences(text):
    alphabets= "([A-Za-z])"
    prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
    suffixes = "(Inc|Ltd|Jr|Sr|Co)"
    starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
    acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
    websites = "[.](com|net|org|io|gov)"

    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
#     text = re.sub("\.{3,}","<ellipsis>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = re.sub("(?<!\.)\.(?!\.)",".<stop>",text) # Avoid taking ellipsis as period
    text = re.sub("\!(?!\!)","!<stop>", text)
    text = re.sub("\?(?!\?)","?<stop>", text)
#     text = text.replace(".",".<stop>")
#     text = text.replace("?","?<stop>")
#     text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [11]:
def extract_sentence(df_posts):
    col_names = ['sent','id','gender','age','industry','sign']
    list_df_sent = []
    
    for _, data in df_posts.iterrows():
        label_list = data[data.index != 'post'].to_list()
        sent_list = split_into_sentences(data['post'])
        df_sent_post = pd.DataFrame(combine_text_label(sent_list, label_list), columns=col_names)
        list_df_sent.append(df_sent_post)
    return pd.concat(list_df_sent).reset_index()

In [15]:
def combine_text_label(post_list, label_list):
    return [[post] + label_list for post in post_list]

In [16]:
df_sents = extract_sentence(df_posts)

In [ ]:
df_sents.drop('index', axis=1, inplace=True)

In [ ]:
df_sents.head()

In [ ]:
df_sents.to_pickle(base_data_path + '/sent_df.pickle')